In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import pandas as pd
import urllib3
import pymysql as ms
from datetime import date

In [2]:
url='https://www.latlong.net/countries.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')
c=soup.find('ul')
links = []
for link in c.find_all('a'):
    links.append('https://www.latlong.net'+link.get('href'))
    print(links)

['https://www.latlong.net/country/afghanistan-1.html']
['https://www.latlong.net/country/afghanistan-1.html', 'https://www.latlong.net/country/aland-islands-2.html']
['https://www.latlong.net/country/afghanistan-1.html', 'https://www.latlong.net/country/aland-islands-2.html', 'https://www.latlong.net/country/albania-3.html']
['https://www.latlong.net/country/afghanistan-1.html', 'https://www.latlong.net/country/aland-islands-2.html', 'https://www.latlong.net/country/albania-3.html', 'https://www.latlong.net/country/algeria-4.html']
['https://www.latlong.net/country/afghanistan-1.html', 'https://www.latlong.net/country/aland-islands-2.html', 'https://www.latlong.net/country/albania-3.html', 'https://www.latlong.net/country/algeria-4.html', 'https://www.latlong.net/country/american-samoa-5.html']
['https://www.latlong.net/country/afghanistan-1.html', 'https://www.latlong.net/country/aland-islands-2.html', 'https://www.latlong.net/country/albania-3.html', 'https://www.latlong.net/country/

In [3]:
connection = ms.connect(
    host='localhost', # 127.0.0.1
    user='root',
    password ='11112002Vasifwww@',
    database='olkeler1',
)

In [4]:
create_table = """
create table places (
   id int(11) not null auto_increment,
   place_name varchar(400) DEFAULT NULL,
   Latitude float DEFAULT NULL,
   Longitude float DEFAULT NULL,
   temperature int(11) DEFAULT NULL,
   feels_like int(11) DEFAULT NULL,
   low int(11) DEFAULT NULL,
   high int(11) DEFAULT NULL,
   wind_speed_ms int(11) DEFAULT NULL,
   humidity_percent int(11) DEFAULT NULL,
   Dew_Pt int(11) DEFAULT NULL,
   UV_Index int(11) DEFAULT NULL,
   Visibility_km int(11) DEFAULT NULL,
   Pressure_hPa int(11) DEFAULT NULL,
   wind_rotation int(11) DEFAULT NULL,
   date_now varchar(400) DEFAULT NULL,
   primary key (id)
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_table)
create_table = """
create table objects (
    id int(11) not null auto_increment,
    objects varchar(400) DEFAULT NULL,
    primary key (id)
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_table)
create_table = """
create table country (
   id int(11) not null auto_increment,
   country varchar(400) DEFAULT NULL,
   primary key (id)
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_table)
create_table = """
create table weather (
   id int(11) not null auto_increment,
   weather varchar(400) DEFAULT NULL,
   primary key (id)
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_table)    

In [5]:
len(links)

225

In [6]:
for link in links[:10]:
    print(link)

https://www.latlong.net/country/afghanistan-1.html
https://www.latlong.net/country/aland-islands-2.html
https://www.latlong.net/country/albania-3.html
https://www.latlong.net/country/algeria-4.html
https://www.latlong.net/country/american-samoa-5.html
https://www.latlong.net/country/andorra-6.html
https://www.latlong.net/country/angola-7.html
https://www.latlong.net/country/anguilla-8.html
https://www.latlong.net/country/antarctica-9.html
https://www.latlong.net/country/antigua-and-barbuda-10.html


In [7]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[:10]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[10:20]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[20:30]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[30:40]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[40:50]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[50:60]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[60:70]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[70:80]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[80:90]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[90:100]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[100:110]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[110:120]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[120:130]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[130:140]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[140:150]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[150:160]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[160:170]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[170:180]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[180:190]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[190:200]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[200:210]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[210:220]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()

In [13]:
latitudes=[]
longlitudes=[]
places1=[]
for link in links[220:]:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    c=soup.find('ul')
    urls = []
    for url in c.find_all('a'):
        urls.append('https://www.latlong.net'+url.get('href'))
    for url in urls:
        timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
        http = urllib3.PoolManager(timeout=timeout)
        no_timeout = urllib3.util.Timeout(connect=None, read=None)
        response = requests.get(url, timeout=no_timeout)
        soup = BeautifulSoup(response.content, 'lxml')
        page_content = soup.find('table')
        items= page_content.find_all('td')
        for item in items[0::3]:
            text0=url.replace('https://www.latlong.net/category/','').split('-')[0]
            insert_sql = 'insert into objects (objects) values(%s)'      
            values=(text0)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)
            connection.commit()
        for item in items[0::3]:
            text2=item.text.split(',')[-1]
            insert_sql = 'insert into country(country) values(%s)'      
            values=(text2)
            cursor = connection.cursor() 
            cursor.execute(insert_sql, values)       
            connection.commit()
        for item in items[0::3]:
            text1=item.text
            places1.append(text1)       
        for item in items[1::3]:
            text2=item.text
            latitudes.append(text2)            
        for item in items[2::3]:
            text1=item.text
            longlitudes.append(text1)
for i in range(len(places1)):
    x=latitudes[i]
    y=longlitudes[i]
    url1='https://darksky.net/forecast/'+x+','+y+'/si12/en'
    timeout = urllib3.util.Timeout(connect=2.0, read=7.0)
    http = urllib3.PoolManager(timeout=timeout)
    no_timeout = urllib3.util.Timeout(connect=None, read=None)
    response = requests.get(url1, timeout=no_timeout)
    urllib3.disable_warnings()
    soup = BeautifulSoup(response.content, 'lxml')
    a=soup.find('div',class_="center")
    z=soup.find('span',class_="direction")
    if a is not None:
        b=a.find('span',class_="summary swap").text
        text1=b.split()[0]
        text1=str(text1).replace('˚','')
        text2=str(b.split()[1:]).replace('.','').replace(']','').replace('[','').replace("'",'').replace(',','')
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (text2)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
        c=soup.find('span',class_="summary-high-low").text
        text3=str(c.split()[2]).replace('˚','')
        text4=str(c.split()[4]).replace('˚','')
        text5=str(c.split()[6]).replace('˚','')   
        d=soup.find('div',id="currentDetails").text
        text6=d.split()[1]
        text67=float(str(str(str(z).split('(')).split(")")).split("'")[4].replace('deg','')) 
        text7=str(d.split()[5]).replace('%','')
        text8=str(d.split()[8]).replace('˚','')
        text9=d.split()[11]
        text10=d.split()[13]
        text11=d.split()[16]
        today = date.today()
        insert_sql = 'insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high,wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation,date_now) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        values = (places1[i],latitudes[i],longlitudes[i],text1,text3,text4,text5,text6,text7,text8,text9,text10,text11,text67,today)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()
    else:
        insert_sql = 'insert into weather (weather) values(%s)'
        values = (1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()   
        insert_sql = """insert into places (place_name,Latitude,Longitude,temperature,feels_like,low,high
        wind_speed_ms,humidity_percent,Dew_Pt,UV_Index,Visibility_km,Pressure_hPa,wind_rotation) 
        values(%s,%s,%s,%s,%s,%s%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000)
        cursor = connection.cursor() 
        cursor.execute(insert_sql, values)
        connection.commit()